# Week 3 assignment.

This is a notebook for Data science Capstone Project assignment for week 3. This notebook includes the following

1. Markdown cell explaining the contents of the notebook
2. Loads data from the wiki page, cleans the data and converts it into a pandas datafram
3. Prints the shape of the data frame


#### Install Required Libraries

In [ ]:
#!pip3 install requests
print("Requests installed...")
#!pip3 install BeautifulSoup4
print("BeautifulSoup installed...")

#### import required Libraries

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### The code below uses requests library to get the content from the wikipedia page and assigns it to variable response. The status code of 200 indicates that the data was successfully received from the wikipedia page.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
response.status_code

200

#### Create a helper method to parse the table and generate a data frame

In [16]:
def parse_table(table):
    n_cols = 0
    n_rows = 0
    col_names = []
    #Determine number of rows and columns in table
    for row in table.find_all('tr'):
        # Determine the number of rows in table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows += 1
            if n_cols == 0:
                # set the number of columns for table
                n_cols = len(td_tags)
        # Determine column names
        th_tags = row.find_all('th')
        if len(th_tags) > 0 and len(col_names) == 0:
            for th in th_tags:
                col_names.append(th.get_text().strip())
    # Create data frame from the table.
    columns = col_names if len(col_names) > 0 else range(0,n_cols)
    df = pd.DataFrame(columns = columns, index = range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        col_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,col_marker] = column.get_text().strip().replace("/",",")
            col_marker +=1
        if len(columns) > 0:
            row_marker +=1
    return df

#### The code below extracts the table from the entire web page data received in response above and calls the helper method to generate a data frame
#### In addition, it filters out the postal code for which the Borough is Not assigned.

In [33]:
soup = BeautifulSoup(response.text,'html')
tables = soup.find('table', {'class':'wikitable'})
table_df = parse_table(tables)
table_df = table_df[table_df.Borough != 'Not assigned']
table_df.head(103)


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


#### Print the shape of the data fram after clean up

In [30]:
table_df.shape

(103, 3)